In [18]:
import pandas as pd
import numpy as np

In [19]:
def enforce_monotonicity(df):
    df = df.copy()

    aps = sorted(df["advanced_purchase"].unique())
    mss = sorted(df["min_stay"].unique())

    pivot = df.pivot(
        index="advanced_purchase",
        columns="min_stay",
        values="fare"
    ).loc[aps, mss]

    # Enforce AP monotonicity
    for i in range(1, len(aps)):
        pivot.iloc[i, :] = np.minimum(
            pivot.iloc[i, :],
            pivot.iloc[i - 1, :]
        )

    # Enforce MS monotonicity
    for j in range(1, len(mss)):
        pivot.iloc[:, j] = np.minimum(
            pivot.iloc[:, j],
            pivot.iloc[:, j - 1]
        )

    return pivot.reset_index().melt(
        id_vars="advanced_purchase",
        var_name="min_stay",
        value_name="fare"
    )

In [ ]:
def fill_fares_grid(fares_df:pd.DataFrame, ap_options: list[int], min_stay_options: list[int]) -> pd.DataFrame:
    """
    fares_df: A dataframe with the columns advanced_purchase, min_stay, and fare.
    ap_options: A list of final ap options we want to have in the grid.
    min_stay_options: A list of fimal min stay options we want to have in the grid.

    Return: A dataframe with the columns advanced_purchase, min_stay, and fare.
    The dataset should have all possible combination of values of advanced_purchase and min_stay.
    Fare column must be filled for each row.
    Complete the missing fares in a way that preserve the "restrictivness rule".
    """
    grid = (
        pd.MultiIndex.from_product(
            [ap_options, min_stay_options],
            names=["advanced_purchase", "min_stay"]
        )
        .to_frame(index=False)
    )
    
    if len(fares_df) == 1:
        ap0 = fares_df.iloc[0]["advanced_purchase"]
        ms0 = fares_df.iloc[0]["min_stay"]
        fare0 = fares_df.iloc[0]["fare"]

        def assign(row):
            ap, ms = row["advanced_purchase"], row["min_stay"]

            if ap == ap0 and ms == ms0:
                return fare0   # exact match
            elif ap >= ap0 and ms >= ms0:
                return fare0 * 0.9   # cheaper, but bounded
            elif ap <= ap0 and ms <= ms0:
                return fare0 * 1.1   # more expensive
            else:
                return fare0         # incomparable → no inference

        grid["fare"] = grid.apply(assign, axis=1)

    else:
        constraints = fares_df.copy()

        # merge exact matches
        grid = grid.merge(
            constraints[['advanced_purchase', 'min_stay', 'fare']], 
            on=['advanced_purchase', 'min_stay'], 
            how='left'
        )

        def compute_bounds(row):
            ap, ms = row["advanced_purchase"], row["min_stay"]
      
            # More restrictive fares (higher AP and MS) = lower fare = lower bound
            more_restrictive = constraints.loc[
                (constraints["advanced_purchase"] >= ap) & (constraints["min_stay"] >= ms), "fare"]

            # Less restrictive fares (lower AP and MS) = higher fare = upper bound
            less_restrictive = constraints.loc[
                (constraints["advanced_purchase"] <= ap) & (constraints["min_stay"] <= ms), "fare"]

            # Lower bound: max of more restrictive fares (highest price among more restrictive = lowest bound)
            lower = more_restrictive.max() if not more_restrictive.empty else np.nan
            # Upper bound: min of less restrictive fares (lowest price among less restrictive = highest bound)
            upper = less_restrictive.min() if not less_restrictive.empty else np.nan

            if not np.isnan(upper) and not np.isnan(lower):
                return (upper + lower) / 2
            elif not np.isnan(upper):
                return upper
            elif not np.isnan(lower):
                return lower
            else:
                return np.nan

        # fill rows without exact matches
        missing_mask = grid['fare'].isna()
        if missing_mask.any():
            grid.loc[missing_mask, 'fare'] = grid[missing_mask].apply(
                compute_bounds, axis=1
            )

    return enforce_monotonicity(grid)
    



In [21]:
def print_prices_grid(result: pd.DataFrame):
    print("Resulting Fare Grid:")
    result_pivot = result.copy()
    result_pivot['min_stay'] = pd.to_numeric(result_pivot['min_stay'])
    result_pivot['advanced_purchase'] = pd.to_numeric(result_pivot['advanced_purchase'])
    print(result_pivot.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())


In [22]:
# Test 1: Single known fare
def test_single_fare():
    fares_df = pd.DataFrame({
        'advanced_purchase': [14],
        'min_stay': [7],
        'fare': [100.0]
    })
    ap_options = [7, 14, 28, 60]
    min_stay_options = [3, 5, 7, 14]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    # Check grid completeness
    assert len(result) == len(ap_options) * len(min_stay_options)
    assert set(result['advanced_purchase']) == set(ap_options)
    assert set(result['min_stay']) == set(min_stay_options)
    
    # Check no missing values
    if result['fare'].isna().any():
        print(f"✗ Test 1 FAILED: Found NaN values in result")
        print(f"  NaN count: {result['fare'].isna().sum()}")
        print(f"  Result with NaN:\n{result[result['fare'].isna()]}")
        raise AssertionError(f"Found {result['fare'].isna().sum()} NaN values in fares")
    
    # Verify exact matches are preserved
    for _, known_fare in fares_df.iterrows():
        ap, ms, fare = known_fare['advanced_purchase'], known_fare['min_stay'], known_fare['fare']
        result_fare = result[(result['advanced_purchase'] == ap) & (result['min_stay'] == ms)]['fare'].iloc[0]
        assert result_fare == fare, \
            f"Exact match not preserved: ({ap}, {ms}) should be {fare} but got {result_fare}"
    
    print("✓ Test 1 passed: Single fare case")   
    print_prices_grid(result)

test_single_fare()

✓ Test 1 passed: Single fare case
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  110.0  110.0  100.0  100.0
5                  110.0  110.0  100.0  100.0
7                  110.0  100.0   90.0   90.0
14                 100.0   90.0   90.0   90.0


In [23]:
# Test 2: Multiple known fares
def test_multiple_fares():
    fares_df = pd.DataFrame({
        'advanced_purchase': [7, 14, 28],
        'min_stay': [3, 7, 14],
        'fare': [150.0, 120.0, 80.0]
    })
    ap_options = [7, 14, 28, 60]
    min_stay_options = [3, 5, 7, 14]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    # Check grid completeness
    assert len(result) == len(ap_options) * len(min_stay_options)
    
    # Check no missing values
    assert not result['fare'].isna().any()
    
    # Verify exact matches are preserved
    for _, known_fare in fares_df.iterrows():
        ap, ms, fare = known_fare['advanced_purchase'], known_fare['min_stay'], known_fare['fare']
        result_fare = result[(result['advanced_purchase'] == ap) & (result['min_stay'] == ms)]['fare'].iloc[0]
        assert result_fare == fare, \
            f"Exact match not preserved: ({ap}, {ms}) should be {fare} but got {result_fare}"
    
    print("✓ Test 2 passed: Multiple fares case")
    print_prices_grid(result)


test_multiple_fares()

TypeError: fill_fares_grid.<locals>.compute_bounds() takes 1 positional argument but 2 were given

In [ ]:
# Test 3: Monotonicity enforcement (restrictiveness rule)
def test_monotonicity():
    fares_df = pd.DataFrame({
        'advanced_purchase': [7, 60],
        'min_stay': [3, 14],
        'fare': [100.0, 80.0]
    })
    ap_options = [7, 14, 28, 60]
    min_stay_options = [3, 5, 7, 14]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    # Sort by restrictiveness (higher AP and MS = more restrictive = lower fare)
    result_sorted = result.sort_values(['advanced_purchase', 'min_stay'])
    
    # Check monotonicity along advanced_purchase (for same min_stay)
    for ms in min_stay_options:
        ms_rows = result[result['min_stay'] == ms].sort_values('advanced_purchase')
        fares = ms_rows['fare'].values
        # More restrictive (higher AP) should have lower or equal fare
        assert all(fares[i] >= fares[i+1] for i in range(len(fares)-1)), \
            "Fares should decrease or stay same as advanced_purchase increases"
    
    # Check monotonicity along min_stay (for same advanced_purchase)
    for ap in ap_options:
        ap_rows = result[result['advanced_purchase'] == ap].sort_values('min_stay')
        fares = ap_rows['fare'].values
        # More restrictive (higher MS) should have lower or equal fare
        assert all(fares[i] >= fares[i+1] for i in range(len(fares)-1)), \
            "Fares should decrease or stay same as min_stay increases"
    
    # Verify exact matches are preserved
    for _, known_fare in fares_df.iterrows():
        ap, ms, fare = known_fare['advanced_purchase'], known_fare['min_stay'], known_fare['fare']
        result_fare = result[(result['advanced_purchase'] == ap) & (result['min_stay'] == ms)]['fare'].iloc[0]
        assert result_fare == fare, \
            f"Exact match not preserved: ({ap}, {ms}) should be {fare} but got {result_fare}"
    
    print("✓ Test 3 passed: Monotonicity enforcement")
    print_prices_grid(result)

test_monotonicity()

✓ Test 3 passed: Monotonicity enforcement
Resulting Fare Grid:
advanced_purchase     7     14    28    60
min_stay                                  
3                  100.0  90.0  90.0  90.0
5                   90.0  90.0  90.0  90.0
7                   90.0  90.0  90.0  90.0
14                  90.0  90.0  90.0  80.0


/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/3242637060.py:35: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())


In [ ]:
# Test 4: Known fares at grid boundaries
def test_boundary_fares():
    fares_df = pd.DataFrame({
        'advanced_purchase': [7, 60],  # Min and max AP
        'min_stay': [3, 14],  # Min and max MS
        'fare': [150.0, 70.0]
    })
    ap_options = [7, 14, 28, 60]
    min_stay_options = [3, 5, 7, 14]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    # Check that known fares are preserved (or adjusted for monotonicity)
    # The least restrictive (7, 3) should have highest fare
    least_restrictive = result[(result['advanced_purchase'] == 7) & 
                               (result['min_stay'] == 3)]['fare'].iloc[0]
    assert least_restrictive >= 140.0, "Least restrictive should have high fare"
    
    # The most restrictive (60, 14) should have lowest fare
    most_restrictive = result[(result['advanced_purchase'] == 60) & 
                              (result['min_stay'] == 14)]['fare'].iloc[0]
    assert most_restrictive <= 80.0, "Most restrictive should have low fare"
    
    # Verify exact matches are preserved
    for _, known_fare in fares_df.iterrows():
        ap, ms, fare = known_fare['advanced_purchase'], known_fare['min_stay'], known_fare['fare']
        result_fare = result[(result['advanced_purchase'] == ap) & (result['min_stay'] == ms)]['fare'].iloc[0]
        assert result_fare == fare, \
            f"Exact match not preserved: ({ap}, {ms}) should be {fare} but got {result_fare}"
    
    print("✓ Test 4 passed: Boundary fares")
    print_prices_grid(result)

test_boundary_fares()

✓ Test 4 passed: Boundary fares
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  150.0  110.0  110.0  110.0
5                  110.0  110.0  110.0  110.0
7                  110.0  110.0  110.0  110.0
14                 110.0  110.0  110.0   70.0


/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/530921301.py:27: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())


In [ ]:
# Test 5: Single option case
def test_single_options():
    fares_df = pd.DataFrame({
        'advanced_purchase': [14],
        'min_stay': [7],
        'fare': [100.0]
    })
    ap_options = [14]
    min_stay_options = [7]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    assert len(result) == 1
    assert result['advanced_purchase'].iloc[0] == 14
    assert result['min_stay'].iloc[0] == 7
    assert not result['fare'].isna().any()
    
    # Verify exact match is preserved
    for _, known_fare in fares_df.iterrows():
        ap, ms, fare = known_fare['advanced_purchase'], known_fare['min_stay'], known_fare['fare']
        result_fare = result[(result['advanced_purchase'] == ap) & (result['min_stay'] == ms)]['fare'].iloc[0]
        assert result_fare == fare, \
            f"Exact match not preserved: ({ap}, {ms}) should be {fare} but got {result_fare}"
    
    print("✓ Test 5 passed: Single option case")
    print_prices_grid(result)

test_single_options()

✓ Test 5 passed: Single option case
Resulting Fare Grid:
advanced_purchase     14
min_stay                
7                  100.0


/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/3905596621.py:21: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())


In [ ]:
# Test 6: Fares that may violate restrictiveness rule (should be corrected)
def test_violating_fares():
    # This has a more restrictive fare that's more expensive (violates rule)
    fares_df = pd.DataFrame({
        'advanced_purchase': [7, 60],
        'min_stay': [3, 14],
        'fare': [100.0, 120.0]  # More restrictive (60, 14) is more expensive - wrong!
    })
    ap_options = [7, 14, 28, 60]
    min_stay_options = [3, 5, 7, 14]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    # After enforce_monotonicity, the most restrictive should not be more expensive
    least_restrictive = result[(result['advanced_purchase'] == 7) & 
                               (result['min_stay'] == 3)]['fare'].iloc[0]
    most_restrictive = result[(result['advanced_purchase'] == 60) & 
                              (result['min_stay'] == 14)]['fare'].iloc[0]
    
    assert least_restrictive >= most_restrictive, \
        "Monotonicity should enforce least_restrictive >= most_restrictive"
    
    # Note: Exact matches may be adjusted by monotonicity enforcement if they violate the rule
    # For this test, we check that the exact match that doesn't violate monotonicity is preserved
    # The fare at (7, 3) = 100.0 should be preserved since it's less restrictive
    less_restrictive_match = result[(result['advanced_purchase'] == 7) & 
                                   (result['min_stay'] == 3)]['fare'].iloc[0]
    assert less_restrictive_match == 100.0, \
        f"Less restrictive exact match should be preserved: got {less_restrictive_match} instead of 100.0"
    
    print("✓ Test 6 passed: Violating fares corrected")
    print_prices_grid(result)

test_violating_fares()

✓ Test 6 passed: Violating fares corrected
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  100.0  100.0  100.0  100.0
5                  100.0  100.0  100.0  100.0
7                  100.0  100.0  100.0  100.0
14                 100.0  100.0  100.0  100.0


/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/12073373.py:26: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())


In [ ]:
# Test 7: Complete grid with known fare in middle
def test_complete_grid():
    # Known fare in the middle of the grid
    fares_df = pd.DataFrame({
        'advanced_purchase': [28],
        'min_stay': [7],
        'fare': [100.0]
    })
    ap_options = [7, 14, 28, 60]
    min_stay_options = [3, 5, 7, 14]
    
    result = fill_fares_grid(fares_df, ap_options, min_stay_options)
    
    # Verify all combinations exist
    expected_combinations = len(ap_options) * len(min_stay_options)
    assert len(result) == expected_combinations
    
    # Verify no duplicates
    assert result.duplicated(subset=['advanced_purchase', 'min_stay']).sum() == 0
    
    # All fares should be filled
    assert result['fare'].notna().all()
    
    # Check that known fare point exists
    known_fare_row = result[(result['advanced_purchase'] == 28) & 
                            (result['min_stay'] == 7)]
    assert len(known_fare_row) == 1
    
    # Verify exact match is preserved
    for _, known_fare in fares_df.iterrows():
        ap, ms, fare = known_fare['advanced_purchase'], known_fare['min_stay'], known_fare['fare']
        result_fare = result[(result['advanced_purchase'] == ap) & (result['min_stay'] == ms)]['fare'].iloc[0]
        assert result_fare == fare, \
            f"Exact match not preserved: ({ap}, {ms}) should be {fare} but got {result_fare}"
    
    print("✓ Test 7 passed: Complete grid generation")
    print_prices_grid(result)

test_complete_grid()

✓ Test 7 passed: Complete grid generation
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  110.0  110.0  110.0  100.0
5                  110.0  110.0  110.0  100.0
7                  110.0  110.0  100.0   90.0
14                 100.0  100.0   90.0   90.0


/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/1647224851.py:32: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())


In [ ]:
# Run all tests
print("\n" + "="*50)
print("Running all tests...")
print("="*50 + "\n")

test_single_fare()
test_multiple_fares()
test_monotonicity()
test_boundary_fares()
test_single_options()
test_violating_fares()
test_complete_grid()

print("\n" + "="*50)
print("All tests completed!")
print("="*50)


Running all tests...

✓ Test 1 passed: Single fare case
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  110.0  110.0  100.0  100.0
5                  110.0  110.0  100.0  100.0
7                  110.0  100.0   90.0   90.0
14                 100.0   90.0   90.0   90.0
✓ Test 2 passed: Multiple fares case
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  150.0  135.0  115.0  115.0
5                  135.0  135.0  115.0  115.0
7                  135.0  120.0  100.0  100.0
14                 115.0  100.0   80.0   80.0
✓ Test 3 passed: Monotonicity enforcement
Resulting Fare Grid:
advanced_purchase     7     14    28    60
min_stay                                  
3                  100.0  90.0  90.0  90.0
5                   90.0  90.0  90.0  90.0
7                   90.0  90.0  90.0  90.0
14                  90.0  90.0  90

/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/2269574501.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())
/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/2398386894.py:22: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())
/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/3242637060.py:35: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())
/var/folde

In [ ]:
# Diagnostic: Check which tests fail
import traceback

tests_to_run = [
    ("Test 1: Single fare", test_single_fare),
    ("Test 2: Multiple fares", test_multiple_fares),
    ("Test 3: Monotonicity", test_monotonicity),
    ("Test 4: Boundary fares", test_boundary_fares),
    ("Test 5: Single options", test_single_options),
    ("Test 6: Violating fares", test_violating_fares),
    ("Test 7: Complete grid", test_complete_grid),
]

print("="*60)
print("Testing all functions...")
print("="*60 + "\n")

failed_tests = []
passed_tests = []

for test_name, test_func in tests_to_run:
    try:
        test_func()
        passed_tests.append(test_name)
    except AssertionError as e:
        print(f"✗ {test_name} FAILED (AssertionError)")
        print(f"  Error: {e}\n")
        failed_tests.append((test_name, str(e)))
    except Exception as e:
        print(f"✗ {test_name} FAILED (Exception)")
        print(f"  Error: {type(e).__name__}: {e}\n")
        traceback.print_exc()
        failed_tests.append((test_name, str(e)))

print("\n" + "="*60)
print(f"Summary: {len(passed_tests)} passed, {len(failed_tests)} failed")
print("="*60)

if failed_tests:
    print("\nFailed tests:")
    for test_name, error in failed_tests:
        print(f"  - {test_name}: {error}")

Testing all functions...

✓ Test 1 passed: Single fare case
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  110.0  110.0  100.0  100.0
5                  110.0  110.0  100.0  100.0
7                  110.0  100.0   90.0   90.0
14                 100.0   90.0   90.0   90.0
✓ Test 2 passed: Multiple fares case
Resulting Fare Grid:
advanced_purchase     7      14     28     60
min_stay                                     
3                  150.0  135.0  115.0  115.0
5                  135.0  135.0  115.0  115.0
7                  135.0  120.0  100.0  100.0
14                 115.0  100.0   80.0   80.0
✓ Test 3 passed: Monotonicity enforcement
Resulting Fare Grid:
advanced_purchase     7     14    28    60
min_stay                                  
3                  100.0  90.0  90.0  90.0
5                   90.0  90.0  90.0  90.0
7                   90.0  90.0  90.0  90.0
14                  90.0  90.0 

/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/2269574501.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())
/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/2398386894.py:22: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())
/var/folders/_9/pd8k82rj1gs2w1lfc1_cvvb40000gn/T/ipykernel_17476/3242637060.py:35: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  print(result.pivot(index='min_stay', columns='advanced_purchase', values='fare').sort_index())
/var/folde